In [2]:
import sys
import os
import math
import ROOT as r
r.gStyle.SetOptStat(0)
r.gStyle.SetOptTitle(0)
r.gStyle.SetLegendBorderSize(0)

import glob
import json

# --------------------------------------------------------------------
# 1. Define cross sections (pb) for each signal file
# --------------------------------------------------------------------
xsec_map = {
    "test_Higgsino_100_50_simple": 0.07668,
    "test_Higgsino_100_80_simple": 0.07298,
    "test_Higgsino_100_90_simple": 0.07148,
    "test_Higgsino_100_95_simple": 0.07070,
    "test_Higgsino_100_98_simple": 0.07031,
    "test_Higgsino_100_99_simple": 0.07023,
    "test_Higgsino_120_70_simple": 0.05649,
    "test_Higgsino_120_100_simple": 0.05303,
    "test_Higgsino_120_110_simple": 0.05168,
    "test_Higgsino_120_115_simple": 0.05096,
    "test_Higgsino_120_118_simple": 0.05059,
    "test_Higgsino_120_119_simple": 0.05045,
    "test_Higgsino_140_90_simple": 0.03699,
    "test_Higgsino_140_120_simple": 0.03331,
    "test_Higgsino_140_130_simple": 0.03194,
    "test_Higgsino_140_135_simple": 0.03131,
    "test_Higgsino_140_138_simple": 0.03087,
    "test_Higgsino_140_139_simple": 0.03071,
    "test_Higgsino_160_110_simple": 0.01806,
    "test_Higgsino_160_140_simple": 0.01411,
    "test_Higgsino_160_150_simple": 0.01271,
    "test_Higgsino_160_155_simple": 0.01201,
    "test_Higgsino_160_158_simple": 0.01163,
    "test_Higgsino_160_159_simple": 0.01149,
    "test_Higgsino_180_130_simple": 0.004725,
    "test_Higgsino_180_160_simple": 0.0009906,
    "test_Higgsino_180_170_simple": 0.0001484,
    "test_Higgsino_182_132_simple": 0.004181,
    "test_Higgsino_182_162_simple": 0.0005962,
    "test_Higgsino_182_172_simple": 0.000005557,
}

# --------------------------------------------------------------------
# 2. Define integrated luminosity (ab^-1) and convert to pb^-1
# --------------------------------------------------------------------
luminosity_ab = 1.0  # 1 ab^-1
luminosity_pb_inv = luminosity_ab * 1e6  # => 1.0e6 pb^-1

# --------------------------------------------------------------------
# 3. Fill the BACKGROUND histograms (with background weight)
#     Make sure they are NOT owned by the file (SetDirectory(0)).
# --------------------------------------------------------------------
hist_H_BG = r.TH1F("m_recoil_BG", "Recoil Mass (Background)", 100, 50, 250)
hist_H_BG.SetDirectory(0)
hist_H_BG.Sumw2()

hist_Z_BG = r.TH1F("m_Z_BG", "Z Mass (Background)", 100, 50, 250)
hist_Z_BG.SetDirectory(0)
hist_Z_BG.Sumw2()

hist_photon_energy_BG = r.TH1F("ph_E_BG", "Photon Energy (Background)", 100, 0, 150)
hist_photon_energy_BG.SetDirectory(0)
hist_photon_energy_BG.Sumw2()

BG_sigma_pb = 2.641
BGfile = r.TFile("/data/mhance/FCCee/test_background_fccee_simple.root", "READ")
BGtree = BGfile.Get("outputTree")

N_gen_bg = BGtree.GetEntries()
weight_bg = (BG_sigma_pb * luminosity_pb_inv) / N_gen_bg

print(f"Background cross section: {BG_sigma_pb} pb")
print(f"Luminosity: {luminosity_ab} ab^-1  (== {luminosity_pb_inv} pb^-1)")
print(f"N_gen (BG): {N_gen_bg}")
print(f"Background weight: {weight_bg:.4e}\n")

ecms = r.TLorentzVector()
ecms.SetPxPyPzE(0, 0, 0, 240)

# Fill the background histograms
for event in BGtree:
    # Photon energies
    for photon_energy in event.ph_E:
        hist_photon_energy_BG.Fill(photon_energy, weight_bg)

    # Only consider events with exactly 2 muons
    if len(event.mu_pt) != 2:
        continue

    mu1 = r.TLorentzVector()
    mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
    mu2 = r.TLorentzVector()
    mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

    Z = mu1 + mu2
    hist_Z_BG.Fill(Z.M(), weight_bg)

    recoil = ecms - Z
    hist_H_BG.Fill(recoil.M(), weight_bg)

BGfile.Close()
print("Finished filling background. Now processing signal...\n")

# --------------------------------------------------------------------
# 4. Process all SIGNAL files in the directory
# --------------------------------------------------------------------
signal_files = glob.glob("/data/mhance/FCCee/test_Higgsino_*_simple.root")
signal_files.sort()

for idx, sig_file in enumerate(signal_files):
    print(f"===================================================")
    print(f"Processing signal file #{idx}: {sig_file}")
    
    base_name = os.path.basename(sig_file)
    base_no_ext = os.path.splitext(base_name)[0]

    # Parse masses from filename, e.g. 'test_Higgsino_100_50_simple.root'
    parts = base_name.split('_')
    m_N2 = int(parts[2])  # e.g., 100
    m_N1 = int(parts[3])  # e.g., 50
    delta_m = m_N2 - m_N1

    if base_no_ext not in xsec_map:
        print(f"WARNING: No cross section found for {base_no_ext}. Skipping.")
        continue

    sigma_pb = xsec_map[base_no_ext]
    infile = r.TFile(sig_file, "READ")
    intree = infile.Get("outputTree")
    N_gen_sig = intree.GetEntries()

    weight_sig = (sigma_pb * luminosity_pb_inv) / N_gen_sig

    print(f"  Cross section    : {sigma_pb} pb")
    print(f"  N_gen (signal)   : {N_gen_sig}")
    print(f"  Signal weight    : {weight_sig:.4e}")
    print(f"  Masses => N2={m_N2} GeV, N1={m_N1} GeV, deltaM={delta_m} GeV")

    # Create unique histograms for this signal, also detached:
    hist_H_signal = r.TH1F(f"m_recoil_signal_{idx}", "Recoil Mass (Signal)", 100, 50, 250)
    hist_H_signal.SetDirectory(0)
    hist_H_signal.Sumw2()

    hist_Z_signal = r.TH1F(f"m_Z_signal_{idx}", "Z Mass (Signal)", 100, 50, 250)
    hist_Z_signal.SetDirectory(0)
    hist_Z_signal.Sumw2()

    hist_photon_energy_signal = r.TH1F(f"ph_E_signal_{idx}", "Photon Energy (Signal)", 100, 0, 150)
    hist_photon_energy_signal.SetDirectory(0)
    hist_photon_energy_signal.Sumw2()

    # Fill signal hists with the computed weight
    for event in intree:
        for photon_energy in event.ph_E:
            hist_photon_energy_signal.Fill(photon_energy, weight_sig)

        if len(event.mu_pt) != 2:
            continue

        mu1 = r.TLorentzVector()
        mu1.SetPtEtaPhiM(event.mu_pt[0], event.mu_eta[0], event.mu_phi[0], event.mu_m[0])
        mu2 = r.TLorentzVector()
        mu2.SetPtEtaPhiM(event.mu_pt[1], event.mu_eta[1], event.mu_phi[1], event.mu_m[1])

        Z = mu1 + mu2
        hist_Z_signal.Fill(Z.M(), weight_sig)

        recoil = ecms - Z
        hist_H_signal.Fill(recoil.M(), weight_sig)

    infile.Close()

    # Compare signal vs. background in a Photon Energy plot
    canvas_photon_E = r.TCanvas(f"photon_energy_canvas_{idx}", f"Photon Energy Canvas {idx}", 800, 600)
    hist_photon_energy_signal.SetLineColor(r.kGreen + 2)  # example color
    hist_photon_energy_signal.Draw("hist")
    hist_photon_energy_BG.SetLineColor(r.kBlue)
    hist_photon_energy_BG.Draw("hist same")

    maxbin = max(
        hist_photon_energy_BG.GetBinContent(hist_photon_energy_BG.GetMaximumBin()),
        hist_photon_energy_signal.GetBinContent(hist_photon_energy_signal.GetMaximumBin())
    )
    hist_photon_energy_signal.GetYaxis().SetRangeUser(0.5, 10*maxbin)
    canvas_photon_E.SetLogy(1)
    canvas_photon_E.SaveAs(f"PhotonEnergy_plots_{idx}.png")

    # Write histograms to a ROOT file
    # Note: we open a NEW root file each iteration. The BG hists are still in memory.
    outroot = r.TFile(f"output_{idx}.root", "RECREATE")
    hist_H_signal.Write()
    hist_Z_signal.Write()
    hist_photon_energy_signal.Write()
    # Also store the BG hists in the same file (optional)
    hist_H_BG.Write()
    hist_Z_BG.Write()
    hist_photon_energy_BG.Write()
    outroot.Close()

    # Optionally, export to JSON
    data_dict = {
        "m_N2": m_N2,
        "delta_m": delta_m,
        "hist_H_signal": {"bins": [], "entries": []},
        "hist_Z_signal": {"bins": [], "entries": []},
        "hist_photon_energy_signal": {"bins": [], "entries": []},
        "hist_H_BG": {"bins": [], "entries": []},
        "hist_Z_BG": {"bins": [], "entries": []},
        "hist_photon_energy_BG": {"bins": [], "entries": []},
    }

    # Fill the dictionaries from histograms
    hist_map = {
        "hist_H_signal": hist_H_signal,
        "hist_Z_signal": hist_Z_signal,
        "hist_photon_energy_signal": hist_photon_energy_signal,
        "hist_H_BG": hist_H_BG,
        "hist_Z_BG": hist_Z_BG,
        "hist_photon_energy_BG": hist_photon_energy_BG
    }

    for hname, th1 in hist_map.items():
        nbins = th1.GetNbinsX()
        for bin_num in range(1, nbins + 1):
            bin_center = th1.GetXaxis().GetBinCenter(bin_num)
            bin_content = th1.GetBinContent(bin_num)
            data_dict[hname]["bins"].append(bin_center)
            data_dict[hname]["entries"].append(bin_content)

    with open(f"extracted_data_{idx}.json", "w") as jf:
        json.dump(data_dict, jf, indent=4)

    print(f"Finished signal file #{idx}: {sig_file}")

print("\nAll signal files processed.")


Background cross section: 2.641 pb
Luminosity: 1.0 ab^-1  (== 1000000.0 pb^-1)
N_gen (BG): 10000
Background weight: 2.6410e+02

Finished filling background. Now processing signal...

Processing signal file #0: /data/mhance/FCCee/test_Higgsino_100_50_simple.root
  Cross section    : 0.07668 pb
  N_gen (signal)   : 10000
  Signal weight    : 7.6680e+00
  Masses => N2=100 GeV, N1=50 GeV, deltaM=50 GeV
Finished signal file #0: /data/mhance/FCCee/test_Higgsino_100_50_simple.root
Processing signal file #1: /data/mhance/FCCee/test_Higgsino_100_80_simple.root
  Cross section    : 0.07298 pb
  N_gen (signal)   : 10000
  Signal weight    : 7.2980e+00
  Masses => N2=100 GeV, N1=80 GeV, deltaM=20 GeV
Finished signal file #1: /data/mhance/FCCee/test_Higgsino_100_80_simple.root
Processing signal file #2: /data/mhance/FCCee/test_Higgsino_100_90_simple.root
  Cross section    : 0.07148 pb
  N_gen (signal)   : 9997
  Signal weight    : 7.1501e+00
  Masses => N2=100 GeV, N1=90 GeV, deltaM=10 GeV
Finishe

Warning in <TCanvas::Constructor>: Deleting canvas with same name: photon_energy_canvas_0
Info in <TCanvas::Print>: png file PhotonEnergy_plots_0.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_1.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_2.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_3.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_4.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_5.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_6.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_7.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_8.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_9.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_10.png has been created
Info in <TCanvas::Print>: png file PhotonEnergy_plots_11.png h